# Import

In [50]:
import pandas as pd 
import plotly as px
pd.set_option('display.max_rows', 500)  # Zeilenanzahl erhöhen
pd.set_option('display.max_columns', 50)  # Spaltenanzahl erhöhen
pd.set_option('display.width', 1000)  # Maximale Breite der Anzeige
import plotly.io as pio


# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template


In [51]:
years = [str(year) for year in range(2013, 2025)]
df = pd.read_excel("data.xlsx", sheet_name=years)
data_list = []
for year, df in df.items():
    # Füge das Jahr als neue Spalte hinzu
    df['Jahr'] = year
    
    # Füge den DataFrame der Liste hinzu
    data_list.append(df)

# Alle DataFrames in der Liste zu einem einzigen DataFrame zusammenführen
final_df = pd.concat(data_list, ignore_index=True)


In [52]:
final_df = final_df[final_df.columns[~final_df.columns.str.contains(r'\*')]] # drop * in the data
final_df.columns = final_df.columns.str.replace('*', '', regex=False)# just to be sure
final_df = final_df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo

clean_columns = {}
for col in final_df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

In [53]:
df = final_df[final_df['Category']=='Anzahl Studierende'].drop(columns=['Variable', 'Category'])
df.columns

Index(['Bachelor Agrarwissenschaften', 'Bachelor Ernährungswissenschaften', 'Bachelor Nachwachsende Rohstoffe und Bioressourcen', 'Bachelor Ökotrophologie', 'Bachelor Umwelt und globaler Wandel', 'Master Ernährungswissenschaften ', 'Master  Agrarökonomie und Betriebsmanagement', 'Master Nutztierwissenschaften', 'Master Pflanzenproduktion', 'Master Haushalts- und Dienstleistungswissenschaften\n', 'Master Umwelt- und Ressourcenmanagement', 'Master Getränketechnologie', 'Jahr', 'Master Getränketechnologie', 'Master Agrar- und Ressourcenökonomie', 'Master Umweltwissenschaften', 'Master Transition Management\n', 'Master Ökotrophologie', 'Master Agrobiotechnologie\n', 'Master Insect Biotechnology and Bioresources', 'Bachelor Umweltmanagement', 'Master Agrobiotechnologie', 'Master Ernährungsökonomie', 'Master Nutzpflanzenwissenschaften', 'Master Oenologie/Weinwirtschaft', 'Master Transition Management', 'Master Nachhaltige Ernährungswirtschaft', 'Master Sustainable Transition'], dtype='object

In [54]:
df_grouped = df.groupby(by='Jahr').sum().reset_index().sort_values(by='Jahr')

In [55]:
df_grouped.sort_values(by='Jahr')
columns_to_plot = df_grouped.columns.to_list()

In [56]:
df

,Bachelor Agrarwissenschaften,Bachelor Ernährungswissenschaften,Bachelor Nachwachsende Rohstoffe und Bioressourcen,Bachelor Ökotrophologie,Bachelor Umwelt und globaler Wandel,Master Ernährungswissenschaften,Master Agrarökonomie und Betriebsmanagement,Master Nutztierwissenschaften,Master Pflanzenproduktion,Master Haushalts- und Dienstleistungswissenschaften\n,Master Umwelt- und Ressourcenmanagement,Master Getränketechnologie,Jahr,Master Getränketechnologie,Master Agrar- und Ressourcenökonomie,Master Umweltwissenschaften,Master Transition Management\n,Master Ökotrophologie,Master Agrobiotechnologie\n,Master Insect Biotechnology and Bioresources,Bachelor Umweltmanagement,Master Agrobiotechnologie,Master Ernährungsökonomie,Master Nutzpflanzenwissenschaften,Master Oenologie/Weinwirtschaft,Master Transition Management,Master Nachhaltige Ernährungswirtschaft,Master Sustainable Transition
66,364.0,251.0,NaN,450.0,341.0,260.0,37.0,40.0,49.0,72.0,166.0,20.0,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,378.0,278.0,NaN,552.0,372.0,223.0,35.0,41.0,55.0,39.0,178.0,NaN,2014,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204,381.0,324.0,NaN,601.0,404.0,213.0,NaN,55.0,75.0,NaN,NaN,NaN,2015,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
273,368.0,348.0,NaN,644.0,435.0,38.0,NaN,172.0,65.0,NaN,NaN,NaN,2016,190.0,52.0,54.0,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
342,385.0,376.0,NaN,650.0,437.0,45.0,NaN,167.0,63.0,NaN,NaN,NaN,2017,69.0,103.0,180.0,54.0,88.0,103.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,335.0,427.0,NaN,673,451.0,319.0,NaN,71.0,94.0,NaN,NaN,62.0,2018,NaN,48.0,170.0,90.0,673.0,111.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
480,374.0,438.0,NaN,656.0,NaN,318.0,NaN,73.0,NaN,NaN,NaN,98.0,2019,NaN,45.0,166.0,NaN,117.0,NaN,25.0,476.0,124.0,181.0,98.0,48.0,96.0,NaN,NaN
549,427.0,475.0,NaN,649.0,NaN,320.0,NaN,67.0,NaN,NaN,NaN,52.0,2020,NaN,49.0,161.0,NaN,141.0,NaN,37.0,469.0,120.0,160.0,97.0,NaN,117.0,NaN,NaN
618,402.0,474.0,27.0,619.0,NaN,346.0,NaN,65.0,NaN,NaN,NaN,44.0,2021,NaN,49.0,161.0,NaN,183.0,NaN,66.0,456.0,138.0,153.0,93.0,NaN,132.0,NaN,NaN
687,367.0,438.0,68.0,566.0,NaN,353.0,NaN,63.0,NaN,NaN,NaN,44.0,2022,NaN,45.0,164.0,NaN,186.0,NaN,85.0,435.0,130.0,127.0,82.0,NaN,205.0,NaN,NaN


### Plot

### Heatmap

In [57]:
import plotly.graph_objects as go
import plotly.express as px
df = df_grouped

# Um die Heatmap zu erstellen, nehmen wir die Spalten außer der "Jahr" Spalte:
heatmap_data = df.drop("Jahr", axis=1)

# Erstelle die Heatmap mit Plotly Express
fig = px.imshow(heatmap_data, 
                labels=dict(x="Studiengang", y="Jahr"), 
                x=heatmap_data.columns, 
                y=df["Jahr"], 
                color_continuous_scale='viridis')  # Du kannst die Farbskala ändern, z.B. 'Cividis', 'Blues', etc.
fig.update_layout(
    width=2500,   # Breite in Pixel
    height=1100    # Höhe in Pixel
)
fig.update_layout(
    xaxis=dict(
        title='Studiengänge',  # Titel der X-Achse (optional)
        tickfont=dict(size=20)  # Schriftgröße der Kategorien auf der X-Achse (Jahre)
    ),
    yaxis=dict(
        title='Jahre',  # Titel der Y-Achse (optional)
        tickfont=dict(size=20)  # Schriftgröße der Kategorien auf der Y-Achse (falls auch für Y-Achse relevant)
    ),
    coloraxis_colorbar=dict(
        tickfont=dict(size=20),
        len=0.7,  # Länge des Farbbalkens auf der Achse
    

))
# Zeige die Heatmap an
fig.show()


In [58]:
bachelor_columns = [col for col in df.columns if "Bachelor" in col]
master_columns = [col for col in df.columns if "Master" in col]

# Erstelle einen DataFrame für Bachelor und aggregiere die Anzahl pro Jahr
bachelor_df = df[["Jahr"] + bachelor_columns].melt(id_vars="Jahr", value_name="Anzahl")
bachelor_df["Studiengang"] = "Bachelor"
bachelor_agg = bachelor_df.groupby(["Jahr", "Studiengang"])["Anzahl"].sum().reset_index()

# Erstelle einen DataFrame für Master und aggregiere die Anzahl pro Jahr
master_df = df[["Jahr"] + master_columns].melt(id_vars="Jahr", value_name="Anzahl")
master_df["Studiengang"] = "Master"
master_agg = master_df.groupby(["Jahr", "Studiengang"])["Anzahl"].sum().reset_index()

# Füge die beiden DataFrames zusammen
final_df = pd.concat([bachelor_agg, master_agg], ignore_index=True).sort_values('Jahr')


In [68]:
fig = px.line(final_df, x="Jahr", y="Anzahl", color='Studiengang',title='Absoluter Rücklauf Bachelor vs Master', template="infoviz", color_discrete_sequence=px.colors.qualitative.Safe)
fig.update_layout(
    yaxis=dict(title="Personen"),)
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px

# ✅ Excel-Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# ✅ Daten für das Jahr 2024 extrahieren
df_2024 = sheets["2024"]

# 🔹 Extrahiere alle Fragen mit "Anzahl" (Anzahl der Antworten)
fragen_anzahl = df_2024[df_2024["Category"] == "Anzahl"]

df_2024[df_2024["Category"] == "Anzahl"]

,Variable,Category,Bachelor Agrarwissenschaften,Bachelor Ernährungswissenschaften,Bachelor Nachwachsende Rohstoffe und Bioressourcen,Bachelor Ökotrophologie,Bachelor Umwelt und globaler Wandel,Master Agrar- und Ressourcenökonomie,Master Agrobiotechnologie,Master Ernährungswissenschaften,...,Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,Master Insect Biotechnology and Bioresources,Master Nachhaltige Ernährungswirtschaft,Master Nutzpflanzenwissenschaften,Master Nutztierwissenschaften*,Master Oenologie/Weinwirtschaft*,Master Ökotrophologie,Master Sustainable Transition,Master Transition Management,Master Umweltwissenschaften
3,Geschlecht,Anzahl,23,63.0,12.0,44.0,52.0,8.0,27.0,33.0,...,NaN,10.0,11.0,3.0,NaN,NaN,22.0,5.0,19.0,17.0
13,Alter,Anzahl,23,63.0,13.0,44.0,53.0,8.0,26.0,33.0,...,NaN,9.0,9.0,3.0,NaN,NaN,22.0,5.0,19.0,17.0
17,Staatsangehörigkeit,Anzahl,26,66.0,13.0,45.0,53.0,9.0,28.0,33.0,...,NaN,10.0,11.0,3.0,NaN,NaN,22.0,5.0,19.0,17.0
21,Migrationshintergrund,Anzahl,23,62.0,12.0,44.0,53.0,8.0,26.0,33.0,...,NaN,10.0,8.0,3.0,NaN,NaN,22.0,4.0,18.0,17.0
37,Fachsemester,Anzahl,34,101.0,18.0,57.0,63.0,13.0,14.0,32.0,...,NaN,10.0,12.0,5.0,NaN,NaN,23.0,5.0,13.0,20.0
44,Zufriedenheit,Anzahl,30,88.0,14.0,51.0,60.0,12.0,33.0,36.0,...,NaN,12.0,13.0,6.0,NaN,NaN,24.0,6.0,22.0,18.0
49,Subjektive Leistungseinschätzung,Anzahl,28,75.0,14.0,48.0,54.0,10.0,26.0,33.0,...,NaN,10.0,12.0,6.0,NaN,NaN,22.0,6.0,20.0,15.0


In [20]:
# 🔹 Gesamtzahl der Studierenden extrahieren
gesamt_row = df_2024[(df_2024["Variable"] == "Rücklaufquote") & (df_2024["Category"] == "Anzahl Studierende")]
gesamt_studierende = gesamt_row.iloc[:, 2:].sum().sum()  # Summe aller Spalten ab der 2. Spalte

gesamt_row


,Variable,Category,Bachelor Agrarwissenschaften,Bachelor Ernährungswissenschaften,Bachelor Nachwachsende Rohstoffe und Bioressourcen,Bachelor Ökotrophologie,Bachelor Umwelt und globaler Wandel,Master Agrar- und Ressourcenökonomie,Master Agrobiotechnologie,Master Ernährungswissenschaften,...,Master Informationstechnologie in den Agrar- und Umweltwissenschaften*,Master Insect Biotechnology and Bioresources,Master Nachhaltige Ernährungswirtschaft,Master Nutzpflanzenwissenschaften,Master Nutztierwissenschaften*,Master Oenologie/Weinwirtschaft*,Master Ökotrophologie,Master Sustainable Transition,Master Transition Management,Master Umweltwissenschaften
66,Rücklaufquote,Anzahl Studierende,293,493.0,69.0,429.0,116.0,53.0,139.0,338.0,...,13.0,81.0,51.0,65.0,41.0,32.0,152.0,59.0,192.0,13.0


In [ ]:

# 🔹 Berechnung der Rücklaufquote pro Frage
ruecklaufquote_fragen = {"Frage": [], "Rücklaufquote (%)": []}

for _, row in fragen_anzahl.iterrows():
    frage = row["Variable"]
    antworten_anzahl = row.iloc[2:].sum()  # Summe der Antworten für diese Frage

    # Rücklaufquote berechnen
    ruecklaufquote = (antworten_anzahl / gesamt_studierende) * 100 if gesamt_studierende > 0 else 0

    # Werte speichern
    ruecklaufquote_fragen["Frage"].append(frage)
    ruecklaufquote_fragen["Rücklaufquote (%)"].append(ruecklaufquote)

# ✅ DataFrame erstellen und nach Rücklaufquote sortieren
df_ruecklauf_fragen = pd.DataFrame(ruecklaufquote_fragen)
df_ruecklauf_fragen = df_ruecklauf_fragen.sort_values(by="Rücklaufquote (%)", ascending=False)

In [30]:

# 🔹 Bar-Chart für die Rücklaufquote pro Frage erstellen
fig = px.bar(
    df_ruecklauf_fragen,
    y="Rücklaufquote (%)",
    x="Frage",
    #orientation="v",  # Horizontal für bessere Lesbarkeit
    title="Rücklaufquote pro Frage (2024)",
    template="infoviz",
    color_discrete_sequence=px.colors.qualitative.Safe
)

# Layout-Anpassungen
fig.update_layout(
    #xaxis=dict(title="Rücklaufquote (%)", range=[0, 20]),
    yaxis=dict(title="Frage"),
    #height=800  # Höhere Darstellung für bessere Lesbarkeit
)

fig.show()

In [33]:
import pandas as pd
import plotly.express as px

# ✅ Excel-Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# 🔹 Dictionary zur Speicherung der Rücklaufquoten pro Frage über die Jahre
ruecklaufquote_fragen = {}

# ✅ Durch alle Jahre iterieren
for jahr, df in sheets.items():
    # 🔹 Fragen mit "Anzahl" extrahieren (Anzahl der Antworten)
    fragen_anzahl = df[df["Category"] == "Anzahl"]

    # 🔹 Gesamtzahl der Studierenden extrahieren
    gesamt_row = df[(df["Variable"] == "Rücklaufquote") & (df["Category"] == "Anzahl Studierende")]
    gesamt_studierende = gesamt_row.iloc[:, 2:].sum().sum()  # Summe aller Spalten ab der 2. Spalte

    # 🔹 Falls keine Gesamtstudierenden vorhanden sind, überspringen
    if gesamt_studierende == 0 or gesamt_row.empty:
        continue

    # 🔹 Berechnung der Rücklaufquote pro Frage für dieses Jahr
    for _, row in fragen_anzahl.iterrows():
        frage = row["Variable"]
        antworten_anzahl = row.iloc[2:].sum()  # Summe der Antworten für diese Frage

        # Rücklaufquote berechnen
        ruecklaufquote = (antworten_anzahl / gesamt_studierende) * 100 if gesamt_studierende > 0 else 0

        # Speichern der Werte in Dictionary (Frage als Schlüssel, Liste der Werte)
        if frage not in ruecklaufquote_fragen:
            ruecklaufquote_fragen[frage] = []
        ruecklaufquote_fragen[frage].append(ruecklaufquote)

# ✅ Durchschnittliche Rücklaufquote pro Frage berechnen
df_ruecklauf_fragen = pd.DataFrame([
    {"Frage": frage, "Durchschnittliche Rücklaufquote (%)": sum(werte) / len(werte)}
    for frage, werte in ruecklaufquote_fragen.items()
])

# 🔹 Nach Rücklaufquote sortieren
df_ruecklauf_fragen = df_ruecklauf_fragen.sort_values(by="Durchschnittliche Rücklaufquote (%)", ascending=False)

# 🔹 Bar-Chart für die durchschnittliche Rücklaufquote pro Frage erstellen
fig = px.bar(
    df_ruecklauf_fragen,
    x="Durchschnittliche Rücklaufquote (%)",
    y="Frage",
    orientation="h",  # Horizontal für bessere Lesbarkeit
    title="Durchschnittliche Rücklaufquote pro Frage (über alle Jahre)",
    template="infoviz"
)

# Layout-Anpassungen
fig.update_layout(
    xaxis=dict(title="Durchschnittliche Rücklaufquote (%)", range=[0, 20]),
    yaxis=dict(title="Frage"),
    height=800  # Höhere Darstellung für bessere Lesbarkeit
)

fig.show()